<a href="https://colab.research.google.com/github/kriskamal/Business-Decision-Modeling/blob/main/Project_1_Group_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 - Group 4: Diet Plan for Kentucky Fried Chicken (KFC)
**OPIM 5641-MS40: Business Decision Modeling - University of Connecticut**

**Team 4:** Yiyao Qiu, Yuqing Li, Jared Bergantino, Kamal Kannan Krishnan

---------------------------------------------------------------------------


**Project description:** In this project, the team is required to build the linear optimization models for a nutrition information dataset of meals in KFC to find out a healthy meal plan.

**Dataset:**  KFC Nutrition Facts & Calorie Information containing:


*   Calories
*   Protein(g)
*   Total Carbs(g)
*   Sodium(mg)
*   Sugars(g)
*   Weight Watchers
*   Food Category

**Project objective:** Satisfy nutritional requirements while minimizing Weight Watcher points.










# Mount Drive and Setup Environment

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import modules
import pandas as pd

In [ ]:
# install GLPK solver
%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

from pyomo.environ import *

SOLVER = 'glpk'
EXECUTABLE = '/usr/bin/glpsol'

# Read Data

In [ ]:
# import dataset
df = pd.read_csv('/content/drive/Shared drives/OPIM-5641-Business Decision Modeling-Group 4/Project one/Kentucky Fried Chicken (KFC) dataset - Kentucky Fried Chicken (KFC) dataset.csv')
df.head()

,Meal,Calories,Protein (g),Total Carbs (g),Sodium (mg),Sugars (g),Weight Watchers,Category
0,Original Recipe Chicken- Whole Wing,140,11,5,450,0,4,Chicken
1,Original Recipe Chicken- Breast,320,36,13,1130,0,8,Chicken
2,Original Recipe Chicken-Breast without skin or...,130,29,0,520,0,3,Chicken
3,Original Recipe Chicken- Drumstick,120,11,3,380,0,3,Chicken
4,Original Recipe Chicken- Thigh,290,18,8,850,0,8,Chicken


In [ ]:
# read column names
df.columns

Index(['Meal', 'Calories', 'Protein (g)', 'Total Carbs (g)', 'Sodium (mg)',
       'Sugars (g)', 'Weight Watchers', 'Category'],
      dtype='object')

# Model 1 - Low-Carb

## Low-Carb Diet

A low-carb diet is a diet that limits the intake of carbohydrates from the meals and encourage a high consumption of protein. It is a popular diet plan for people who want to lose the weight. Additionally, according to an article from Mayo Clinic, this diet plan may have other healthy benefits like reducing risk factors associated with type 2 diabetes and metabolic syndrome.

**The daily intake of carbohydrates is typically limited to $60$ grams per day.** 

Source: https://www.mayoclinic.org/healthy-lifestyle/weight-loss/in-depth/low-carb-diet/art-20045831

## Other Constraints of Nutrition We Selected

Besides daily carbohydrate intake, we also find other constraints for sugar, sodium and calories. All contraints are geared towards nutrient recommendations for adult women.


*   **For sugar**, the American Heart Association
(AHA) suggests that **women should consume no more than $25$ grams or $100$ calories** of added sugar per day. 
** Source: https://www.heart.org/en/healthy-living/healthy-eating/eat-smart/sugar/how-much-sugar-is-too-much#:~:text=To%20keep%20all%20of%20this,or%20100%20calories)%20per%20day
*   **For sodium**, the ideal limit recommended from AHA is **less than $1,500$ mg per day**.
** Source: https://www.heart.org/en/healthy-living/healthy-eating/eat-smart/sodium/sodium-and-salt?gclid=CjwKCAjw5p_8BRBUEiwAPpJO69pzjLTHZH8Uah7NvZDSnSAWxrEfkPJdMtYdAkViDO7G0ftc1l3hyRoCU-QQAvD_BwE
*  **For calories**, according to Harvard Medical School "calorie intake should not fall below 1,200 a day in women...eating too few calories can endanger your health by depriving you of needed nutrients." For our project, we based calorie intake off of women, thus our contraint for calories was that **an individual must consume at least $1,200$ calories**.
** Source: https://www.health.harvard.edu/staying-healthy
calorie-counting-made-easy







## Extract data list for modeling

In [ ]:
# extract data from dataset for modeling

# create list of meal indices
meal_index = df.index.values.tolist()
print('meal index', meal_index)

# create dictionary of meal 
meal_name =  df['Meal'].T.to_dict()
print(meal_name)

# create list of nutrients values - from Calories to Sugars(g)
calories = df['Calories'].tolist()
carb = df['Total Carbs (g)'].tolist()
sodium = df['Sodium (mg)'].tolist()
sugar = df['Sugars (g)'].tolist()

# create list of weight watchers
weight_watchers = df['Weight Watchers'].tolist()
print('weight watchers', weight_watchers)

meal index [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162]
{0: 'Original Recipe Chicken- Whole Wing', 1: 'Original Recipe Chicken- Breast', 2: 'Original Recipe Chicken-Breast without skin or breading', 3: 'Original Recipe Chicken- Drumstick', 4: 'Original Recipe Chicken- Thigh', 5: 'Original Recipe Boneless Piece (White Meat)', 6: 'Original

## Declare model, objective and constraints

*   **Objective Function**

  $min Weight Watchers$

*   **Write the Constraints**
  
  $Calories \geq 1200$

  $Carb \leq 60$

  $Sodium \leq 1500$

  $Sugar \leq 25$

  $Carb, Sodium, Sugar \in \mathbb{R}^+$ `(Domains)`








In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.volume = Var(meal_index, domain=NonNegativeIntegers)

# declare objective
obj_expr = 0
for meal in meal_index:
  obj_expr += weight_watchers[meal]*model.volume[meal] # Objective to minimize Weight Watchers points
model.weightwatchers = Objective(
                      expr = obj_expr,
                      sense = minimize)

# declare constraints
cal_exp = 0
for meal in meal_index:
  cal_exp += calories[meal]*model.volume[meal] # Calorie Constraint
model.calorie = Constraint(expr = cal_exp >= 1200)

carb_exp = 0
for meal in meal_index:
   carb_exp += carb[meal]*model.volume[meal] # Carb Constraint
model.carb = Constraint(expr = carb_exp <= 60)

sod_exp = 0
for meal in meal_index:
  sod_exp += sodium[meal]*model.volume[meal] # Sodium Constraint
model.sodium = Constraint(expr = sod_exp <= 1500)

sug_exp = 0
for meal in meal_index:
  sug_exp += sugar[meal]*model.volume[meal] # Sugar Constraint
model.sugar = Constraint(expr = sug_exp <= 25)

# show the model 
model.pprint()

1 Set Declarations
    volume_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  163 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162}

1 Var Declarations
    volume : Size=163, Index=volume_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
         

## Solve the problem and show the results

In [ ]:
# solve it
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model).write()
# show the results
print('\nWeight Watcher Points=', model.weightwatchers())
print('calories=', model.calorie())
print('carb=', model.carb())
print('sodium=', model.sodium())
print('sugar=', model.sugar())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 28.0
  Upper bound: 28.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 164
  Number of nonzeros: 533
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 97
      Number of created subproblems: 97
  Error rc: 0
  Time: 0.02177143096923828
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [ ]:
# show the results of selected meals in dataframe

# create list of each meal and volume from the model
meal_volume_list = []
def get_meal_volume_list(meal_index):
  for meal in meal_index:
    meal_volume_list.append([meal_name[meal],model.volume[meal]()])

get_meal_volume_list(meal_index)

# transfer list to dataframe
df_meal = pd.DataFrame(meal_volume_list, columns = ['Meal', 'Volume'])  

# show the selected meal results with nutrition
df_selected_meal = df_meal[df_meal['Volume']!=0.0]
result = pd.merge(df_selected_meal, df, how='inner',on=['Meal'])
result

,Meal,Volume,Calories,Protein (g),Total Carbs (g),Sodium (mg),Sugars (g),Weight Watchers,Category
0,House Side Salad without Dressing,8.0,15,1,3,10,2,0,Salads and More
1,Heinz Buttermilk Ranch Dressing (1),6.0,160,0,1,220,1,4,Salads and More
2,Corn on the Cob (3″ ),1.0,70,2,16,0,3,2,Sides (Individual)
3,Colonel’s Buttery Spread,2.0,30,0,0,30,0,1,Sides (Individual)


# Model 2 - Moderate-Carb

## Moderate-Carb Diet

We decided to alter our initial diet after conducting more research into the benefits of limiting carbs. Specifically, in a dietary carbohydrate study, doctors discovered that "in the meta-analysis of all cohorts (432 179 participants), both low carbohydrate consumption (<40%) and high carbohydrate consumption (>70%) conferred greater mortality risk than did moderate intake." As a result, we decided to increase the carb contraints based on information from the Dietary Guidelines for Americans (DGA). 

The DGA recommends that **carbohydrates make up $45$ to $65$ percent of the total daily calorie intake**. People need a minimum of $1,200$ calories daily to stay healthy. And according to Calorie Calculator from Mayo Clinic, estimated calories intake for women are:

* Inactive: $1600$
* Somewhat Active: $1750$
* Active: $1900$

Based on average women's body measurements:
* Height in inches: $63.6$
* Weight in pounds: $170.5$
* Waist circumference in inches: $38.7$

**Sources:** 
* Moderate carb intake: https://www.thelancet.com/journals/lanpub/article/PIIS2468-2667(18)30135-X/fulltext
* DGA article: https://www.mayoclinic.org/healthy-lifestyle/weight-loss/in-depth/low-carb-diet/art-20045831
* Average women's body measurements:https://www.cdc.gov/nchs/fastats/body-measurements.htm



## Other Constraints of Nutrition We Selected

Besides daily carbohydrate intake, we also find other constraints for sugar, sodium and protein.


*   **For sugar**, the American Heart Association
(AHA) suggests that **women should consume no more than $25$ grams or $100$ calories** of added sugar per day. 
*   **For sodium**, the ideal limit recommended from AHA is **less than $1,500$ mg per day**. 
*   **For protein**, the current international Recommended Dietary Allowance (RDA) for protein is **minimum $0.8$ grams per kg of body weight**. For reference, according to statistics from Centers for Disease Control and Prevention (CDC), the average weight for women aged 20 and over is 170.5lb(77.3kg). Therefore, **the minimum intake of protein for adult women is around $62$ grams**.


**Sources:**
* Sugar intake: https://www.heart.org/en/healthy-living/healthy-eating/eat-smart/sugar/how-much-sugar-is-too-much#:~:text=To%20keep%20all%20of%20this,or%20100%20calories)%20per%20day
* Sodium intake: https://www.heart.org/en/healthy-living/healthy-eating/eat-smart/sodium/sodium-and-salt?gclid=CjwKCAjw5p_8BRBUEiwAPpJO69pzjLTHZH8Uah7NvZDSnSAWxrEfkPJdMtYdAkViDO7G0ftc1l3hyRoCU-QQAvD_BwE
*  Protein intake: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5872778/








## Declare model, objective and constraints

*   **Objective Function**

  $min Weight Watchers$

*   **Constraints**

  $1200 \leq Calories \leq 1900$

  $Protein \geq 62$

  $ 135 \leq Carb \leq 214$ (for reference: from $(minCal*0.45)/4$ to $(minCal*0.65)/4$ )

  $Sodium \leq 1500$

  $Sugar \leq 25$

  $Sodium, Sugar \in \mathbb{R}^+$ `(Domains)`









In [ ]:
# extract data from dataset for modeling

# create list of meal indices
meal_index = df.index.values.tolist()
print('meal index', meal_index)

# create dictionary of meal 
meal_name =  df['Meal'].T.to_dict()
print(meal_name)

# create list of nutrients values - from Calories to Sugars(g)
calories = df['Calories'].tolist()
protein = df['Protein (g)'].tolist()
carb = df['Total Carbs (g)'].tolist()
sodium = df['Sodium (mg)'].tolist()
sugar = df['Sugars (g)'].tolist()

# create list of weight watchers
weight_watchers = df['Weight Watchers'].tolist()
print('weight watchers', weight_watchers)

meal index [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162]
{0: 'Original Recipe Chicken- Whole Wing', 1: 'Original Recipe Chicken- Breast', 2: 'Original Recipe Chicken-Breast without skin or breading', 3: 'Original Recipe Chicken- Drumstick', 4: 'Original Recipe Chicken- Thigh', 5: 'Original Recipe Boneless Piece (White Meat)', 6: 'Original

In [ ]:
# declare the model
model = ConcreteModel()

# declare decision variables
model.volume = Var(meal_index, domain=NonNegativeIntegers, bounds=(0,3))

# declare objective
obj_expr = 0
for meal in meal_index:
  obj_expr += weight_watchers[meal]*model.volume[meal] # Objective to minimize Weight Watchers points
model.weightwatchers = Objective(
                      expr = obj_expr,
                      sense = minimize)

#declare constraints
def cal_rule(model):
 return 1200 <= sum(calories[meal]*model.volume[meal] for meal in meal_index) <= 1900 # Calorie Constraint
model.calorie = Constraint(rule=cal_rule)

prot_exp = 0
for meal in meal_index:
  prot_exp += protein[meal]*model.volume[meal]
model.protein = Constraint(expr = prot_exp >= 62) # Protein Constraint

def carb_rule(model):
 return 135 <= sum(carb[meal]*model.volume[meal] for meal in meal_index) <= 214 # Carb Constraint
model.carb = Constraint(rule=carb_rule)

sod_exp = 0
for meal in meal_index:
  sod_exp += sodium[meal]*model.volume[meal]
model.sodium = Constraint(expr = sod_exp <= 1500) # Sodium Constraint

sug_exp = 0
for meal in meal_index:
  sug_exp += sugar[meal]*model.volume[meal]
model.sugar = Constraint(expr = sug_exp <= 25) # Sugar Constraint

# show the model 
model.pprint()

    function to express ranged inequality expressions. (called from
    /usr/local/lib/python3.6/dist-packages/pyomo/core/base/util.py:422)
    function to express ranged inequality expressions. (called from
    /usr/local/lib/python3.6/dist-packages/pyomo/core/base/util.py:422)
1 Set Declarations
    volume_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  163 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 13

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


## Solve the problem and show the results

In [ ]:
# solve it
SolverFactory(SOLVER, executable=EXECUTABLE).solve(model).write()
# show the results
print('\nWeight Watcher Points=', model.weightwatchers())
print('calories=', model.calorie())
print('protein=', model.protein())
print('carb=', model.carb())
print('sodium=', model.sodium())
print('sugar=', model.sugar())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 30.0
  Upper bound: 30.0
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 164
  Number of nonzeros: 907
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 7
      Number of created subproblems: 7
  Error rc: 0
  Time: 0.02454686164855957
# ----------------------------------------------------------
#   Solution Information
# -----------------------------------

In [ ]:
# show the results of selected meals in dataframe

# create list of each meal and volume from the model
meal_volume_list = []
def get_meal_volume_list(meal_index):
  for meal in meal_index:
    meal_volume_list.append([meal_name[meal],model.volume[meal]()])

get_meal_volume_list(meal_index)

# transfer list to dataframe
df_meal = pd.DataFrame(meal_volume_list, columns = ['Meal', 'Volume'])  

# show the selected meal results with nutrition
df_selected_meal = df_meal[df_meal['Volume']!=0.0]
result = pd.merge(df_selected_meal, df, how='inner',on=['Meal'])
result

,Meal,Volume,Calories,Protein (g),Total Carbs (g),Sodium (mg),Sugars (g),Weight Watchers,Category
0,Extra Crispy Tenders (1) – Kids,3.0,130,11,6,310,0,3,Strips and Filets
1,House Side Salad without Dressing,3.0,15,1,3,10,2,0,Salads and More
2,Heinz Buttermilk Ranch Dressing (1),1.0,160,0,1,220,1,4,Salads and More
3,Corn on the Cob (3″ ),3.0,70,2,16,0,3,2,Sides (Individual)
4,Sweet Kernel Corn,3.0,100,3,21,0,3,3,Sides (Individual)
5,Sargento Light String Cheese,2.0,50,6,1,160,0,1,Sides (Individual)


# Conclusions

**What we learned:**
1.   Minimization problems require context. In our first model, the code and mathematics powering the model are technically correct; however, the results are not practical. A meal should not contain 8 salads, 6 dressings, a corn on the cob and 2 colonels buttery spreads.
2.   Declaring the constraints within a range (e.g., 1200 <= Calories <= 1900) helps to refine results rather than just using a maximum or minimum constraint.
3. Dieting at a fast food restaurant (specfically, a low carb diet) is very challenging. The optimal results require an individual to eat a variety of items that do not constitute a normal meal. KFC does not offer many "healthy" options to support individuals on a diet.

**Suggestions for future research:**
1.   Splitting the calories between three meals (e.g., breakfast, lunch, dinner) to better segment the final results/balance out the food items for each meal.
2.  Cleaning the underlying data to pull out particularly unhealthy items and impractical food categories (e.g., dressings and beverages) to refine the final food item recommendations.


